In [2]:
import numpy as np
import cv2
import os
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [3]:
tf.keras.backend.clear_session()

In [4]:
# import shutil

# shutil.rmtree('__MACOSX')

In [5]:
!unzip -q "/content/MP_Data.zip"

# Preprocessing Data

In [6]:
# Path for exported data, numpy arrays
DATA_PATH = 'MP_Data'
# Actions that we try to detect
actions = np.array(['good', 'bad', 'iloveyou', 'hello'])
# Thirty videos worth of data
no_sequences = 30
# Videos are going to be 30 frames in length
sequence_length = 30

In [7]:
label_map = {label:num for num, label in enumerate(actions)}

In [8]:
label_map

{'good': 0, 'bad': 1, 'iloveyou': 2, 'hello': 3}

In [9]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [10]:
X = np.array(sequences)

In [11]:
y = to_categorical(labels).astype(int)

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Build Model

In [13]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [14]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [15]:
# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',
#                                                   patience=5)

In [16]:
model.fit(X_train, y_train, epochs=200)

Epoch 1/200
3/3 [==============================] - 6s 88ms/step - loss: 6.2604 - categorical_accuracy: 0.2500
Epoch 2/200
3/3 [==============================] - 0s 86ms/step - loss: 3.4759 - categorical_accuracy: 0.2500
Epoch 3/200
3/3 [==============================] - 0s 95ms/step - loss: 1.4073 - categorical_accuracy: 0.3021
Epoch 4/200
3/3 [==============================] - 0s 76ms/step - loss: 1.4165 - categorical_accuracy: 0.2396
Epoch 5/200
3/3 [==============================] - 0s 81ms/step - loss: 1.3897 - categorical_accuracy: 0.2917
Epoch 6/200
3/3 [==============================] - 0s 80ms/step - loss: 1.3850 - categorical_accuracy: 0.3229
Epoch 7/200
3/3 [==============================] - 0s 78ms/step - loss: 1.3854 - categorical_accuracy: 0.2812
Epoch 8/200
3/3 [==============================] - 0s 86ms/step - loss: 1.3818 - categorical_accuracy: 0.2917
Epoch 9/200
3/3 [==============================] - 0s 75ms/step - loss: 1.3866 - categorical_accuracy: 0.2917
Epoch 10/2

# Confusion Matrix

In [23]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [20]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 57ms/step


In [21]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [24]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[17,  1],
        [ 2,  4]],

       [[14,  2],
        [ 1,  7]],

       [[15,  1],
        [ 1,  7]],

       [[21,  1],
        [ 1,  1]]])

In [25]:
accuracy_score(ytrue, yhat)

0.7916666666666666

# Save Model


In [26]:
model.save('action.h5')